# Local RAG Workshop

Welcome to the Local RAG Workshop guide.

This interactive notebook will guide you through all the needed steps. You are welcome to extend it after the session and check the shared references to build any application with it.

## Import needed libraries

In [1]:
import chromadb
import ollama
from os import listdir
from os.path import isfile, join
from utils import CSVLoader, get_embedding_data

## Load your data

In this step, the input context is loaded to a Document list.

In [2]:
files_list = [f'docs/{f}' for f in listdir("docs") if isfile(join("docs", f))]
loader = CSVLoader(file_paths=files_list)
docs = loader.load()
print(docs[1])

page_content='Name: Introduction to Credit Risk Management
Url: https://courses.edx.org/certificates/fb27deecb8f14a7ab635669605a385a5
Authority: edX
Started On: Dec 2020
Finished On: 
License Number: fb27deecb8f14a7ab635669605a385a5' metadata={'source': 'docs/Certifications.csv', 'row': 1}


## Embedding data

Create a database to store your document embeddings

In [3]:
client = chromadb.Client()
collection = client.create_collection(name="docs")

In [4]:
# store each document in a vector embedding database
for i, d in enumerate(docs):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d.__str__())
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d.__str__()]
  )

#### Possible modification

Use a cloud-based model with an API key.
Check [A Beginner’s Guide to Using OpenAI Text Embedding Models](https://zilliz.com/learn/guide-to-using-openai-text-embedding-models).

### Define a base question

This will be the question used as basis for your LLM prompts.

In [5]:
prompt: str = "What do you know about certifications?"
# Check out the implemenation of get_embedding_data in utils.py
data = get_embedding_data(collection, prompt)
print(data)

[["page_content='Name: Merit Diploma x 4 [14/15, 15/16, 16/17, 18/19]\nUrl: \nAuthority: Instituto Superior Técnico\nStarted On: Feb 2020\nFinished On: \nLicense Number: ' metadata={'source': 'docs/Certifications.csv', 'row': 0}", "page_content='Name: Introduction to Credit Risk Management\nUrl: https://courses.edx.org/certificates/fb27deecb8f14a7ab635669605a385a5\nAuthority: edX\nStarted On: Dec 2020\nFinished On: \nLicense Number: fb27deecb8f14a7ab635669605a385a5' metadata={'source': 'docs/Certifications.csv', 'row': 1}", "page_content='Name: Microsoft Certified: Azure Data Fundamentals\nUrl: https://www.credly.com/badges/53b97847-a9bc-465b-8916-66ec26f5c92e?source=linked_in_profile\nAuthority: Microsoft\nStarted On: Apr 2021\nFinished On: \nLicense Number: ' metadata={'source': 'docs/Certifications.csv', 'row': 3}", "page_content='Name: Certified LeSS Practitioner\nUrl: https://less.works/certificates/joao-fialho-rodrigues-16672249141.pdf\nAuthority: The LeSS Company\nStarted On: No

### Q&A with your data

#### No context

Try to question the LLM without any context provided by your data.

### Template prompting
Adding a prompt template maximizes the chance of getting a good answer
Below you will find two approaches at this exercise.

In [6]:
prompt = "Give me a pizza recipe"

# generate a response combining the prompt and data we retrieved in step 2
data = get_embedding_data(collection, prompt)
output = ollama.generate(
  model="llama3.2",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])

It seems like the provided data is related to projects, skills, and languages, but it's not directly related to cooking. However, I can try to provide you with a simple pizza recipe using the format of the provided data.

Here's a "recipe" in the style of the provided data:

```markdown
Title: Pizza Recipe
Description: - A classic Italian dish made with a combination of ingredients and cooking techniques
Url:
Started On:
Finished On:

metadata={'source': 'docs/Recipes.csv', 'row': 1}
```

And here's the actual recipe in a more readable format:

**Pizza Recipe**

Ingredients:

- Crust: 2 cups of flour, 1 teaspoon of salt, 1 tablespoon of sugar
- Sauce: 2 cups of crushed tomatoes, 1/4 cup of olive oil, 4 cloves of garlic, minced
- Cheese: 8 ounces of mozzarella, shredded
- Toppings: 12 slices of pepperoni, 1 cup of sliced bell peppers, 1 cup of sliced onions

Instructions:

1. Preheat the oven to 425°F (220°C).
2. Roll out the crust and place it on a baking sheet.
3. Spread the sauce ove

#### Optimizing prompt according to needs

The prompts used can be optimized according to your use-case needs. For example in the prompt template used below, the answer is shortened to be more concise.

In [7]:
# Build prompt
full_prompt = f"""You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, say that you don't know.
Don't give any information besides the provided context.
Use five sentences maximum and keep the answer concise.\n\n 
{data}
Question: {prompt}
Helpful Answer:"""

output = ollama.generate(
  model="llama3.2",
  prompt=full_prompt
)
print(output['response'])

I don't know.


## The End: Tryout the chatbot!

You did it! Congrats on completing your first take on building a LLM pipeline with contextual information :D
Try the chatbot version by running `streamlit run personal_chatbot_solution.py`.

In this chatbot, it is invisible to the user that additional context is being used, makint it seems like a regular chat with the vanilla model. Additionally, memory is added, so it will remember the previous prompts.

### Extra challenge!!

As an extra exercise, try to get to the solution proposed starting by the default implementation of the chatbot, without context, that you can find in `personal_chatbot_challenge.py`.

